In [12]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb


In [13]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")


In [14]:
# Drop columns with too many missing values or low relevance
cols_to_drop = ['Alley', 'PoolQC', 'Fence', 'MiscFeature', 'FireplaceQu', 'Id']
train_df.drop(columns=cols_to_drop, inplace=True)
test_df.drop(columns=cols_to_drop, inplace=True)


In [15]:
X = train_df.drop("SalePrice", axis=1)
y = train_df["SalePrice"]

# Combine for consistent preprocessing
X_full = pd.concat([X, test_df], axis=0)


In [16]:
categorical_cols = X_full.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X_full.select_dtypes(include=['int64', 'float64']).columns.tolist()


In [17]:
# Numerical pipeline
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical pipeline
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine both
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, numerical_cols),
    ('cat', categorical_pipeline, categorical_cols)
])


In [18]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', xgb.XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=4, n_jobs=-1))
])

In [19]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)

# Validation RMSE
y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"Validation RMSE: {rmse:.2f}")

Validation RMSE: 25389.21


In [20]:
X_test = test_df.copy()
final_predictions = model.predict(X_test)

In [21]:
# Read the Id column from the original test.csv
test_ids = pd.read_csv("test.csv")["Id"]

submission = pd.DataFrame({
    "Id": test_ids,
    "SalePrice": final_predictions
})

submission.to_csv("submission.csv", index=False)
print("submission.csv created")

submission.csv created
